In [1]:
import pandas as pd
import numpy as np
import surprise
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import accuracy
from surprise.prediction_algorithms import *
from surprise.model_selection import GridSearchCV
import pickle
pd.set_option('display.max_rows', None)
import heapq
from collections import defaultdict
from operator import itemgetter
import random 
%load_ext autotime

In [22]:
df =  pd.read_csv('./Data/clean_data2.csv')

time: 482 ms


In [3]:
reader = Reader(line_format=u'rating user item', sep=',', rating_scale=(1, 5), skip_lines=1)

time: 1 ms


In [23]:
beer_sample = pickle.load(open('./Data/beer_sample.sav', 'rb'))

time: 3 ms


In [24]:
Model = pickle.load(open('./Data/final_model.sav', 'rb'))

time: 4.52 s


In [2]:
def rater(num):
    userID = 'Tye'
    rating_list = []
    while num > 0:

        beer = (random.choice(beer_sample))
        print(beer)
        rating = input("Let's rate some beers on a scale of 1-5, press n if you have not drank it :\n")
        if rating == 'n':
            continue
        else:
            rating_one_beer = {'user':userID,'item':beer,'rating':rating}
            rating_list.append(rating_one_beer) 
            num -= 1
    return rating_list      

time: 6.02 ms


In [7]:
def rec(testSubject, k):
    
    
    

    testUserInnerID =  trainsetfull.to_inner_uid(testSubject)
    similarityRow = simsMatrix[testUserInnerID]

    similarUsers = []
    for innerID, score in enumerate(similarityRow):
        if (innerID != testUserInnerID):
            similarUsers.append( (innerID, score) )
        
    kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])

    candidates = defaultdict(float)
    for similarUser in kNeighbors:
        innerID = similarUser[0]
        userSimilarityScore = similarUser[1]
        theirRatings = trainsetfull.ur[innerID]
        for rating in theirRatings:
            candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore

    watched = {}
    for itemID, rating in trainsetfull.ur[testUserInnerID]:
        watched[itemID] = 1   
    
    pos = 0
    for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
        if not itemID in watched:
            movieID = trainsetfull.to_raw_iid(itemID)
            print(movieID)
            pos += 1
            if (pos >= k):
                break
    return 

time: 2 ms


In [16]:
user = rater(5)

A Little Sumpin' Extra! Ale
How do you rate this beer on a scale of 1-5, press n if you have not drank it :
5
Bell's Hopslam Ale
How do you rate this beer on a scale of 1-5, press n if you have not drank it :
4
Dos Equis Special Lager
How do you rate this beer on a scale of 1-5, press n if you have not drank it :
4
Brooklyn Brown Ale
How do you rate this beer on a scale of 1-5, press n if you have not drank it :
5
Keystone Light
How do you rate this beer on a scale of 1-5, press n if you have not drank it :
2
time: 11 s


In [17]:
new_ratings_df = df.append(user,ignore_index=True)
new_ratings_df.drop(columns = 'Unnamed: 0', inplace =True)
new_ratings_df.columns  

Index(['user', 'item', 'rating'], dtype='object')

time: 224 ms


In [18]:
data = Dataset.load_from_df(new_ratings_df,reader)

time: 837 ms


In [19]:
 trainsetfull = data.build_full_trainset()

time: 664 ms


In [20]:
Model.fit(trainsetfull)
simsMatrix = Model.compute_similarities()

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
time: 3min 59s


In [21]:
testSubject = 'Tye'
rec(testSubject, 5)

Chocolate Stout
Imperial Stout
Ten FIDY
Pale Ale
Saison Dupont
time: 246 ms
